In [1]:
import cv2
import mediapipe as mp
from pynput.keyboard import Controller, Key
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Create smaller buttons for play, pause, next, and previous
button_size = (50, 50)
button_spacing = 20
button_y = 200

play_button = (100, button_y, 100 + button_size[0], button_y + button_size[1])
pause_button = (play_button[2] + button_spacing, button_y, play_button[2] + button_spacing + button_size[0], button_y + button_size[1])
next_button = (pause_button[2] + button_spacing, button_y, pause_button[2] + button_spacing + button_size[0], button_y + button_size[1])
prev_button = (next_button[2] + button_spacing, button_y, next_button[2] + button_spacing + button_size[0], button_y + button_size[1])

# Initialize volume control parameters
volume_control_width = 50
volume_control_y = 200
volume_control_height = 200
volume_control_x = 580 - volume_control_width  # Positioned on the right-most side of a 640x480 screen

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Initialize the keyboard controller
keyboard = Controller()

# Initialize volume level
current_volume = 50  # Start with a default volume (e.g., 50%)

# Initialize variables for press and hold
time_finger_over_button = 0

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB for MediaPipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    results = hands.process(frame_rgb)

    # Check if hands were detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for button, key_press in zip(
                [play_button, pause_button, next_button, prev_button],
                [Key.media_play_pause, Key.media_play_pause, Key.media_next, Key.media_previous]
            ):
                # Convert normalized coordinates to pixel coordinates
                h, w, c = frame.shape
                x1, y1, x2, y2 = button

                # Draw buttons
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

                # Check if the fingertip (e.g., index finger tip) is over the button
                fingertip = hand_landmarks.landmark[8]  # Index finger tip
                cx, cy = int(fingertip.x * w), int(fingertip.y * h)
                if x1 < cx < x2 and y1 < cy < y2:
                    # If the fingertip is over the button, change its color
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), -1)
                    
                    # Increment time_finger_over_button
                    time_finger_over_button += 1

                    # Check for press and hold
                    press_and_hold_threshold = 30  # You may need to fine-tune this value
                    if time_finger_over_button > press_and_hold_threshold:
                        # Simulate continuous key press when the button is pressed and held
                        keyboard.press(key_press)
                    else:
                        # Simulate a single key press when the button is tapped
                        keyboard.press(key_press)
                        keyboard.release(key_press)
                else:
                    # Draw the button with the original color
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

        # Check if the hand is over the volume control area
        if volume_control_x <= cx <= volume_control_x + volume_control_width and volume_control_y <= cy <= volume_control_y + volume_control_height:
            # Calculate the volume level based on the vertical hand position
            volume_range = volume_control_height
            new_volume = int((cy - volume_control_y) / volume_range * 100)
            # Adjust volume within a reasonable range (0 to 100)
            new_volume = max(0, min(new_volume, 100))
            # Determine whether to increase or decrease the volume
            if new_volume > current_volume:
                keyboard.press(Key.media_volume_up)
                keyboard.release(Key.media_volume_up)
            elif new_volume < current_volume:
                keyboard.press(Key.media_volume_down)
                keyboard.release(Key.media_volume_down)
            # Update the current volume
            current_volume = new_volume
        else:
            # Reset time_finger_over_button when the fingertip is not over any button or volume control
            time_finger_over_button = 0

    # Draw the volume control
    cv2.rectangle(frame, (volume_control_x, volume_control_y), (volume_control_x + volume_control_width, volume_control_y + volume_control_height), (0, 0, 255), 2)
    volume_control_fill = int(volume_control_height * (current_volume / 100))
    cv2.rectangle(frame, (volume_control_x, volume_control_y + volume_control_height - volume_control_fill), (volume_control_x + volume_control_width, volume_control_y + volume_control_height), (0, 255, 0), -1)

    # Display the video feed
    cv2.imshow("Music Playback Control", frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
